In [1]:
import segmentation_models as sm
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from skimage.transform import resize
import matplotlib.pyplot as plt
import cv2
from utils import interpolate_image
from config import *

Segmentation Models: using `keras` framework.


In [2]:
sm.set_framework('tf.keras')
sm.framework()

model = sm.Unet(BACKBONE, classes=n_classes, activation=activation,
                input_shape=(input_images_size, input_images_size, 1),
                encoder_weights=None)
optim = tf.keras.optimizers.Adam(learning_rate)

dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5),
           sm.metrics.FScore(threshold=0.5)]

model.compile(optim, total_loss, metrics)

In [3]:
model.load_weights("models/best_model.hdf5")

In [4]:
tf.__version__


'2.6.0'

In [5]:
test_path = "DATA/test_data/images"
test_mask_path = "DATA/test_data/masks"
test_images = os.listdir(test_path)

In [6]:
data = []
for e in test_images:
    img = cv2.imread(f'{test_path}/{e}',0)
    orig = img.copy()
    orig = cv2.resize(orig,(input_images_size,input_images_size))
    img = interpolate_image(img, target_shape)
    img = np.array([img])
    pred = model.predict(img)
    pred = np.round(pred[0]).astype("uint8")
    img = cv2.imread(f'{test_mask_path}/{e}',0)
    test = cv2.resize(img,(input_images_size,input_images_size))
    _,test = cv2.threshold(test, 127, 255, cv2.THRESH_BINARY)
  
    test[test==255]=1
    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(list(test.reshape(1,input_images_size*input_images_size)[0]),list(pred[:, :, 0].reshape(1,input_images_size*input_images_size)[0]))
    data.append(m.result().numpy())
    temp = pred[:, :, 0]
    temp[temp==1]=255
    test[test==1]=255
    stacked = np.hstack((orig,test,temp))
    cv2.imwrite(f'results/{e}',stacked)
    

/home/hassanrehman/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [15]:
a = np.array(data)
print(f"Testing IOU Score of images:{round(np.mean(a)*100)}%")

Testing IOU Score of images:89%
